# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
'''
# columns:
artist_name
song_title
song_length_seconds
session_id
item_in_session
# composite primary key session_id + item_in_session
'''
query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + " (session_id int, item_in_session int, artist_name text, song_title text, song_length_seconds float," + \
        " PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute("DROP TABLE IF EXISTS song_info_by_session")
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist_name, song_title, song_length_seconds)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
try:
    rows = session.execute("SELECT artist_name, song_title, song_length_seconds FROM song_info_by_session WHERE session_id = 338 AND item_in_session = 4")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length_seconds=495.30731201171875)


In [11]:
'''
# columns:
artist_name
song_title
session_id
item_in_session
user_id
user_full_name
# key:
composite primary key: session_id + user_id + item_in_session
    with clustering columns user_id and item_in_session
'''
query = "CREATE TABLE IF NOT EXISTS song_info_by_user_and_session"
query = query + " (session_id int, user_id int, item_in_session int, artist_name text, song_title text, user_full_name text," + \
        " PRIMARY KEY ((session_id, user_id), item_in_session)) WITH CLUSTERING ORDER BY (item_in_session ASC)"
try:
    session.execute("DROP TABLE IF EXISTS song_info_by_user_and_session")
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_user_and_session (session_id, user_id, item_in_session, artist_name, song_title, user_full_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], f"{line[1]} {line[4]}"))

In [13]:
try:
    rows = session.execute("SELECT artist_name, song_title, user_full_name FROM song_info_by_user_and_session" + \
                           " WHERE session_id = 182 AND user_id = 10")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_full_name='Sylvie Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_full_name='Sylvie Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_full_name='Sylvie Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_full_name='Sylvie Cruz')


In [14]:
'''
# columns:
song_title
user_full_name
user_id
# key:
primary key: composite primary key song_title + user_last_name + user_id
    with clustering column user_last_name for convenient ordering
'''
query = "CREATE TABLE IF NOT EXISTS user_by_song_title"
query = query + " (song_title text, user_id int, user_full_name text," + \
        " PRIMARY KEY (song_title, user_id)) WITH CLUSTERING ORDER BY (user_id ASC)"
try:
    session.execute("DROP TABLE IF EXISTS user_by_song_title")
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song_title (song_title, user_id, user_full_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), f"{line[1]} {line[4]}"))

In [16]:
try:
    rows = session.execute("SELECT user_full_name FROM user_by_song_title" + \
                           " WHERE song_title = 'All Hands Against His Own'")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user_full_name='Jacqueline Lynch')
Row(user_full_name='Tegan Levine')
Row(user_full_name='Sara Johnson')


In [17]:
query = "DROP TABLE IF EXISTS song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_info_by_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_by_song_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [18]:
session.shutdown()
cluster.shutdown()